In [78]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [79]:
raw_df = pd.read_excel('D:\jupyter notebooks\sentiment.xlsx', skiprows=1)

# Combine first two rows to create proper column names
col_names = []
for col in range(raw_df.shape[1]):
    col_name_parts = [str(raw_df.iloc[0, col]), str(raw_df.iloc[1, col])]
    # Filter out 'nan' and 'Unnamed' parts
    col_name_parts = [part for part in col_name_parts if part.lower() != 'nan' and 'unnamed' not in part.lower()]
    col_names.append(' '.join(col_name_parts).strip())

# Now read the data skipping the first few rows and apply the combined column names
df = pd.read_excel('D:\jupyter notebooks\sentiment.xlsx', skiprows=6)
df.columns = col_names

# Clean up column names: remove any remaining 'Unnamed' references
df.columns = [col.strip() for col in df.columns]
df.head()

,Reported Date,Bullish,Neutral,Bearish,Total,8-week Mov Avg,Bull-Bear Spread,Bullish Average,Average +St. Dev.,Average - St. Dev.,Weekly High,Weekly Low,Weekly Close
0,1987-07-24 00:00:00,0.36,0.50,0.14,1.0,NaN,0.22,0.376163,0.476929,0.275398,311.39,307.81,309.27
1,1987-07-31 00:00:00,0.26,0.48,0.26,1.0,NaN,0.00,0.376163,0.476929,0.275398,318.66,310.65,318.66
2,1987-08-07 00:00:00,0.56,0.15,0.29,1.0,NaN,0.27,0.376163,0.476929,0.275398,323.00,316.23,323.00
3,1987-08-14 00:00:00,0.45,0.35,0.20,1.0,NaN,0.25,0.376163,0.476929,0.275398,334.65,323.00,333.99
4,1987-08-21 00:00:00,0.66,0.28,0.06,1.0,NaN,0.60,0.376163,0.476929,0.275398,335.90,329.25,335.90


In [80]:
df.rename(columns={"Reported Date": "time"}, inplace=True)

In [81]:
df = df[pd.to_datetime(df['time'], errors='coerce').notna()]
df

,time,Bullish,Neutral,Bearish,Total,8-week Mov Avg,Bull-Bear Spread,Bullish Average,Average +St. Dev.,Average - St. Dev.,Weekly High,Weekly Low,Weekly Close
0,1987-07-24 00:00:00,0.360000,0.500000,0.140000,1.0,NaN,0.220000,0.376163,0.476929,0.275398,311.39,307.81,309.27
1,1987-07-31 00:00:00,0.260000,0.480000,0.260000,1.0,NaN,0.000000,0.376163,0.476929,0.275398,318.66,310.65,318.66
2,1987-08-07 00:00:00,0.560000,0.150000,0.290000,1.0,NaN,0.270000,0.376163,0.476929,0.275398,323.00,316.23,323.00
3,1987-08-14 00:00:00,0.450000,0.350000,0.200000,1.0,NaN,0.250000,0.376163,0.476929,0.275398,334.65,323.00,333.99
4,1987-08-21 00:00:00,0.660000,0.280000,0.060000,1.0,NaN,0.600000,0.376163,0.476929,0.275398,335.90,329.25,335.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,2025-07-24 00:00:00,0.367781,0.291793,0.340426,1.0,0.375055,0.027355,0.376163,0.476929,0.275398,6360.64,6262.27,6358.91
1982,2025-07-31 00:00:00,0.403333,0.266667,0.330000,1.0,0.384647,0.073333,0.376163,0.476929,0.275398,6409.26,6336.38,6362.90
1983,2025-08-07 00:00:00,0.348703,0.219020,0.432277,1.0,0.382401,-0.083574,0.376163,0.476929,0.275398,6427.02,6212.69,6345.06
1984,2025-08-14 00:00:00,0.298817,0.239645,0.461538,1.0,0.378242,-0.162721,0.376163,0.476929,0.275398,6480.28,6310.32,6450.51


In [82]:
df=df.drop(columns=['Total'])

In [83]:
df.columns

Index(['time', 'Bullish', 'Neutral', 'Bearish', '8-week Mov Avg',
       'Bull-Bear Spread', 'Bullish Average', 'Average +St. Dev.',
       'Average - St. Dev.', 'Weekly High', 'Weekly Low', 'Weekly Close'],
      dtype='object')

In [84]:
df.rename(columns={
    'Bullish': 'bullish',
    'Neutral': 'neutral',
    'Bearish': 'bearish',
    '8-week Mov Avg': '8_week_avg',
    'Bull-Bear Spread': 'bull_bear_spread',
    'Bullish Average': 'bullish_avg',
    'Average +St. Dev.': 'bullish_avg_plus_std',
    'Average - St. Dev.': 'bullish_avg_minus_std',
    'Weekly High': 'weekly_high',
    'Weekly Low': 'weekly_low',
    'Weekly Close': 'weekly_close'
}, inplace=True)

In [86]:
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=('Weekly Close vs Bullish Sentiment',
                                  'Bullish Sentiment with Moving Average'),
                    vertical_spacing=0.1,
                    row_heights=[0.4, 0.6],
                    specs=[[{"secondary_y": True}],  # Enable secondary y-axis for first subplot
                          [{"secondary_y": False}]])  # No secondary y-axis for second subplot

# First subplot: Weekly Close and Bullish Sentiment
fig.add_trace(
    go.Scatter(x=df['time'], y=df['weekly_close'],
               name='Weekly Close', line=dict(color='blue')),
    row=1, col=1,
    secondary_y=False  # Use primary y-axis for price
)
fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish'],
               name='Bullish Sentiment', line=dict(color='green')),
    row=1, col=1,
    secondary_y=True  # Use secondary y-axis for sentiment
)
fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish_avg'],
               name='Bullish Average', line=dict(color='red', dash='dash')),
    row=1, col=1,
    secondary_y=True  # Use secondary y-axis for sentiment average
)

# Second subplot: Bullish Sentiment with Moving Average and reference lines
fig.add_trace(
    go.Scatter(x=df['time'], y=df['bullish'],
               name='Bullish Sentiment', line=dict(color='green')),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=df['time'], y=df['8_week_avg'],
               name='8-Week Moving Average', line=dict(color='purple')),
    row=2, col=1
)

# Add reference lines for the second subplot
fig.add_hline(y=df['bullish_avg'].iloc[0], line_dash="dash", 
              line_color="gray", annotation_text="Average",
              row=2, col=1)
fig.add_hline(y=df['bullish_avg_plus_std'].iloc[0], line_dash="dash",
              line_color="rgba(0,255,0,0.5)", annotation_text="Average +StDev",
              row=2, col=1)
fig.add_hline(y=df['bullish_avg_minus_std'].iloc[0], line_dash="dash",
              line_color="rgba(255,0,0,0.5)", annotation_text="Average -StDev",
              row=2, col=1)

# Update layout
fig.update_layout(
    height=1200,
    width=1400,
    title_text="AAII Sentiment Analysis",
    showlegend=True
)

# Update axes titles
fig.update_yaxes(title_text="Price", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Bullish Sentiment %", row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="Bullish %", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

# Update x-axes to show date in a readable format
fig.update_xaxes(tickformat="%Y-%m-%d")

fig.show()

# Print current readings
print("\nCurrent Readings:")
print(f"Weekly Close: ${df['weekly_close'].iloc[0]:.2f}")
print(f"Bullish Sentiment: {df['bullish'].iloc[0]:.1f}%")
print(f"8-Week Moving Average: {df['8_week_avg'].iloc[0]:.1f}%")
print(f"Bullish Average: {df['bullish_avg'].iloc[0]:.1f}%")
print(f"Average +St. Dev: {df['bullish_avg_plus_std'].iloc[0]:.1f}%")
print(f"Average -St. Dev: {df['bullish_avg_minus_std'].iloc[0]:.1f}%")


Current Readings:
Weekly Close: $309.27
Bullish Sentiment: 0.4%
8-Week Moving Average: nan%
Bullish Average: 0.4%
Average +St. Dev: 0.5%
Average -St. Dev: 0.3%
